<a href="https://colab.research.google.com/github/augustompm/course-multiobjective-optimization/blob/main/multiobjetive_optimization_c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-hq3dyuqt
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-hq3dyuqt
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 801584cceb559adc54e828ebe9b385c5f53fe70f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10743 sha256=3a570d3ab74e9051a615aa78d54722c78a69a0c83a1d407fccde14c37a30e96c
  Stored in directory: /tmp/pip-ephem-wheel-cache-1o_3i0__/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [3]:
%load_ext nvcc4jupyter



Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpwmaiywlb".


In [4]:
%%cuda
//Olá mundo imprimindo de 1 até 10
#include <iostream>
int main() {
    for (int i = 1; i <= 10; ++i) {
        std::cout << i << " ";
    }
    std::cout << std::endl;
    return 0;
}


1 2 3 4 5 6 7 8 9 10 



In [5]:
%%cuda
//Olá mundo imprimindo de 1 até 10
#include <iostream>
int main() {
    for (int i = 1; i <= 10; ++i) {
        std::cout << i << " ";
    }
    std::cout << std::endl;
    return 0;
}

1 2 3 4 5 6 7 8 9 10 

